In [1]:
using PyPlot

INFO: Loading help data...


In [47]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

(fig, ax) = plt.subplots(8,8)
x=[1:1:100]
y=[rand(100)]
li=[hi[:plot](x, y)[1] for hi in ax]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:10
    
    #[fig[:canvas][:restore_region](hi) for hi in backgrounds]
    [hi[:set_ydata](y2[i,:]) for hi in li]
    [fig[:canvas][:blit](hi[:bbox]) for hi in ax]
    #fig[:canvas][:draw]()
end

elapsed time: 1.072894188 seconds (471040 bytes allocated)


In [1]:
using SortSpikes, ExtractSpikes

In [10]:

s=[i => SpikeDetection() for i=1:64]
c=[i => Cluster() for i=1:64]
electrode=[i => zeros(Int64,2) for i=1:64]
neuronnum=[i => zeros(Int64,2) for i=1:64]
be=ones(Int64,20000,64)
for i=1:64
    be[200:1000:end,i]=10*be[200:1000:end,i]
end
onlineCal(be,c,s)
@time onlineSort([convert(Int32,1),convert(Int32,20000)],be,c,s,electrode,neuronnum)



  55.765 milliseconds (39238 allocations: 7256 KB)


In [11]:
main()

LoadError: main not defined
while loading In[11], in expression starting on line 1

In [11]:
function takearray(hi::Array{Int64,1},i::Int64)
    length(hi)
end

takearray (generic function with 3 methods)

In [13]:

@time takearray(be[:,1],1)

elapsed time: 3.2267e-5 seconds (16240 bytes allocated)


2000

In [13]:
c[1]

ExtractSpikes.Cluster(50x5 Array{Float64,2}:
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
 10.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  ⋮                      
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0,[20,20,0,0,0],1,4.046152307615297)